In [334]:
# SETUP
using JuMP
using GLPK
using Random
using LinearAlgebra
using Distributions
using Statistics
include("dynamic_functions.jl")

mask (generic function with 1 method)

# no retention yet 
we start with a matching model with learning:
- workers have noisy signal 
- firm remember their true abilities once employed
- firms and workers rematch after each period 

Task: we will graph the total surplus achieved after N periods of switching
ideally we should see the surplus increases as firm learn about the workers 

In [335]:
# set up 
no_w, no_f = 100,5
no_period = 10 #number of period
noisy_level = 1
ability = rand(Normal(0,2), no_w,no_f)

signal = ability .+ rand(Normal(0, noisy_level), size(ability))
# display(ability)
q = [20,20,20,20,20]
for i in 1:no_period
    # display(signal)
    match = solve_model(signal, false, q)
    # display(match)
    update_signal(signal, match)
    # for i in 1:no_w, j in 1:no_f
    #     if match[i,j] == 1
    #         signal[i,j] = ability[i,j] #signal becomes true ablity as firms learn
    #     end
    # end
    println(sum(ability .* match))
end

opt_match = solve_model(ability, false, q)
print("optimal match would yield: ", sum(ability .* opt_match))

#observe the convergence as firm learns

# an idea: how big would the market need to be
# for it to be counterintuitive for firm to keep search 
# factor: market size, noise level, etc. 
# the scale between talent variance and noise level too 


# how about a model where firms can collude
# to let each other know about worker's abilities

218.5412060434488
233.43841620272778
234.85411549389354
235.04807772303468
235.74151786877468
236.0791116761475
236.0791116761475
236.0791116761475
236.0791116761475
236.0791116761475
optimal match would yield: 238.76094364532287

# with retention where firm can fire workers 
1. now we allow for firm to retain workers 
2. once retained workers are employed forever 
3. we can compare convergence between firms with different retention rules

In [344]:
# set up 
no_w, no_f = 100,5
no_period = 20 #number of period
noisy_level = 5
ability = rand(Normal(0,1), no_w,no_f)
signal = ability .+ rand(Normal(0, noisy_level), size(ability))

original = copy(signal) 
Random.seed!(1234)
# q = [20,20,20,20,20]
for i in 1:no_period 
    # matching now 
    match = solve_model(signal, false, q)
    println(sum(match))
    # firm learns about true ability
    update_signal(signal, match) #update with real ability that firm learns
    # display(signal)
    # display decides to retain 
    retention_list = retention(signal, match)
    println("retention_list", retention_list)

    # update signal matrix 
    new_signal = mask(signal, retention_list)
    signal = new_signal
    # now we have an updated matrix of available worker for next matching period 
end 


100.0
retention_listAny[(4, 3), (5, 3), (14, 3), (18, 3), (25, 5), (26, 4), (30, 4), (34, 3), (35, 4), (54, 1), (55, 5), (65, 4), (78, 1), (93, 1)]
77.0
retention_listAny[(1, 1), (4, 3), (6, 3), (9, 2), (10, 2), (11, 1), (12, 2), (18, 2), (20, 5), (23, 4), (28, 4), (30, 4), (34, 2), (43, 2), (45, 2), (46, 1), (54, 5), (55, 4), (56, 4), (57, 5), (59, 1), (61, 2), (65, 3), (68, 5), (77, 2), (80, 3), (83, 2)]
39.0
retention_listAny[(3, 5), (4, 1), (8, 1), (9, 4), (11, 3), (13, 1), (14, 3), (17, 5), (19, 2), (22, 5), (23, 2), (28, 2), (30, 4), (33, 1), (34, 3), (43, 5), (46, 2), (51, 2), (52, 1), (54, 2), (55, 3), (58, 1), (59, 2)]
9.0
retention_listAny[(12, 3), (13, 5), (16, 4), (17, 3), (24, 5), (28, 4), (29, 5), (33, 2)]
1.0
retention_listAny[(21, 5)]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]
0.0
retention_listAny[]

# firm updating beliefs in Bayesian ways 
1. compare between firms with a bad start vs. firms with a good start 
2. firms form beliefs about signal distribution 
3. we look into 20 percentile and 80 percentile of firm with first-period signal beliefs 